[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/ninarina12/ML_PNR/blob/master/pnr_vae.ipynb)

### Only if running in Google Colaboratory:
- Go to Runtime > Change runtime type, and select GPU.
- Clone the GitHub repository to access supplementary files:

In [ ]:
!git clone https://github.com/ninarina12/ML_PNR.git
%cd ML_PNR

### Import packages

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import os, sys
import json
import torch
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader, TensorDataset

#from plot_sld import plot_SLD

from utils.utils_data import (u, parse_metadata, parse_labels, get_exp_names, read_exp, process_data, process_exp,
                              normalize_log, split_data, drop_duplicates_list, convert_units)
from utils.utils_plot import format_axis, fontsize, textsize, cmap, cmap_temp, cmap_cm, cmap_mse, set_colors, palette
from utils.utils_model import (TorchStandardScaler, VAE, CVAE, init_model, train, evaluate,
                               get_predictions, get_predictions_exp)
from utils.utils_analyze import (plot_history, plot_history_statistics, plot_weights, plot_decoded, plot_decoded_exp,
                                 plot_latent_representation, plot_predicted, plot_exp_statistics, get_threshold,
                                 reject_outliers)
from utils.utils_sld import plot_SLD

# reproducibility
seed = 12
torch.set_default_dtype(torch.float64)
torch.manual_seed(seed)
np.random.seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Load and process synthetic data

- Input the `sample_name` and the `set` number of the dataset to locate the appropriate directories and read the metadata.


- Load and process (apply noise, smoothing, and normalization to) spectra and identify the parameter names `y_header` and corresponding indices `y_ids` to use for supervised learning.


- Standardize `y_data` (parameter values), perform train/valid/test split stratified by class, and convert data to torch tensors.

In [ ]:
# set data properties
sample_name = 'BiSe10_EuS5' #'CrO20_BiSbTe20'
dataset = 0
exclude_s = False
exclude_inst = True
add_mt = False
delta = 0.5

# get directories
set_dir = 'results/' + sample_name + '/set_' + str(dataset)
data_dir = set_dir + '/data'

# parse metadata and experiment files
layers, rho, M, N, q_min, q_max = parse_metadata(set_dir)
q = 10*np.linspace(q_min, q_max, N)
exp_names = get_exp_names(sample_name)

print('set directory:', set_dir)

In [ ]:
# load and process data
x_data, x_orig, y_data, y_columns, y_header, y_ids, \
y_labels, y_units, y_units_ = process_data(data_dir, sample_name, q, seed, delta, exclude_s, exclude_inst, add_mt)

# split data
idx_train, idx_valid, idx_test = split_data(x_data, seed)

# log scale and normalize
x_train, x_moms = normalize_log(x_data[idx_train])
x_valid, _ = normalize_log(x_data[idx_valid], x_moms)
x_test, _ = normalize_log(x_data[idx_test], x_moms)

In [ ]:
# convert to torch tensors
x_train = torch.from_numpy(x_train).unsqueeze(1)
x_valid = torch.from_numpy(x_valid).unsqueeze(1)
x_test = torch.from_numpy(x_test).unsqueeze(1)
y_train = torch.from_numpy(y_data[idx_train])
y_valid = torch.from_numpy(y_data[idx_valid])
y_test = torch.from_numpy(y_data[idx_test])

height = x_train.size()[2]
width = x_train.size()[3]
num_features = len(y_ids)
prox_features = [k for k, v in enumerate(y_header) if 'prox' in v]
print('height:', height, 'width:', width, 'num_features:', num_features)

In [ ]:
# standardize data
scaler = TorchStandardScaler()
scaler.fit(y_train[:,y_ids])
y_train[:,np.array(y_ids)] = scaler.transform(y_train[:,y_ids])
y_valid[:,np.array(y_ids)] = scaler.transform(y_valid[:,y_ids])
y_test[:,np.array(y_ids)] = scaler.transform(y_test[:,y_ids])

scaler.mean = scaler.mean.to(device)
scaler.std = scaler.std.to(device)

### Define model

- Input the `model name` (vae, cvae) for the model to train according to:
    * vae: variational autoencoder (VAE)
    * cvae: VAE conditioned on parameters in `y_header`


- Input the `model_num` to resume training (or to continue analysis without further fitting), or -1 to start a new fit.


- Input the number of models (`reps`) to train in succession, and set the `batch_size` and number of `epochs`.

In [ ]:
# model to train
model_name = 'cvae'
model_num = -1            # -1 to start a new fit, model number to resume or analyze

# training parameters
reps = 10
batch_size = 512
epochs = 100
z_std_norm = False

# model parameters
kwargs = {
    'latent_dim': 24,
    'start_filters': 16,
    'kernel_size': 7,
    'pool_size': 4,
    'num_conv': 2,
    'num_dense': 4,
    'slope': 0.2,
    'drop': False,
    'beta_1': 0.01,
    'beta_2': 0.05
}

# print model architecture
model, _, _ = init_model(model_name, height, width, num_features, kwargs, device, scaler=scaler)
print(model)
print('number of parameters:', sum([p.numel() for p in model.parameters() if p.requires_grad]))

In [ ]:
# set model directories
model_prefix = '/' + model_name + '_'
resume_fit = model_num >= 0

if resume_fit:
    # load args from saved model directory
    model_dir = set_dir + model_prefix + str(model_num)
    args = torch.load(model_dir + '/model_0/model.torch', map_location='cpu')['args']
    start_epoch = args['epochs']
    epochs = start_epoch + epochs
    args['epochs'] = epochs
    
    z_std_norm = args['z_std_norm']
    kwargs = args['kwargs']
    
else:
    # create new model directory
    args = {}
    dirs = next(os.walk(set_dir))[1]
    for k in dirs:
        if k.startswith('.'): dirs.remove(k)
    dirs.remove('data')
    if 'properties' in dirs: dirs.remove('properties')

    if len(dirs):
        idns = [int(d.split('_')[-1]) for d in dirs]
        idn = max(idns) + 1
        args['model_num'] = idn
        model_dir = set_dir + model_prefix + str(idn)
    else:
        args['model_num'] = 0
        model_dir = set_dir + model_prefix + '0'

    os.makedirs(model_dir)
    start_epoch = 0
    
    # save arguments
    args['batch_size'] = batch_size
    args['reps'] = reps
    args['epochs'] = epochs
    args['z_std_norm'] = z_std_norm
    args['kwargs'] = kwargs.copy()
    
meta = json.dumps(args, sort_keys=True, indent=2)
with open(model_dir + '/args.txt', 'w') as f:
    f.write(meta)

print('model directory:', model_dir)

In [ ]:
# configure data loaders
data_train = TensorDataset(x_train, y_train)
data_valid = TensorDataset(x_valid, y_valid)
data_test = TensorDataset(x_test, y_test)

train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(data_valid, batch_size=batch_size)
test_loader = DataLoader(data_test, batch_size=batch_size)

d_sets = ['train', 'valid', 'test']
data_loaders = dict(zip(d_sets, [DataLoader(data_train, batch_size=batch_size), valid_loader, test_loader]))

### Train model

- Train reps number of models in succession.

In [ ]:
checkpoint = 5
for r in range(reps):
    image_dir = model_dir + '/model_' + str(r)
    if not os.path.exists(image_dir): os.makedirs(image_dir)

    model, opt, metric_keys = init_model(model_name, height, width, num_features, kwargs, device, scaler=scaler)

    print('====== repetition:', r)

    if resume_fit:
        model.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['state'])
        opt.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['optimizer'])
        dynamics = torch.load(image_dir + '/model.torch', map_location='cpu')['dynamics']
    else: dynamics = []

    for epoch in range(start_epoch + 1, epochs + 1):
        train(epoch, model, opt, metric_keys, train_loader, device, model_name, y_ids, z_std_norm)
        
        if (epoch % checkpoint) == 0:
            print("({}) Training:".format(epoch))
            train_metrics = evaluate(epoch, model, metric_keys, train_loader, device, model_name, y_ids, z_std_norm)
            print("({}) Validation:".format(epoch))
            valid_metrics = evaluate(epoch, model, metric_keys, valid_loader, device, model_name, y_ids, z_std_norm)
            dynamics.append({
                'epoch': epoch,
                'train': train_metrics,
                'valid': valid_metrics
            })

            results = {
                'args': args,
                'dynamics': dynamics,
                'state': model.state_dict(),
                'optimizer': opt.state_dict()
            }

            # save data and metadata to .torch dictionary
            with open(image_dir + '/model.torch', 'wb') as f:
                torch.save(results, f)
    
    # save final results
    results = {
        'args': args,
        'dynamics': dynamics,
        'state': model.state_dict(),
        'optimizer': opt.state_dict()
    }

    # save data and metadata to .torch dictionary
    with open(image_dir + '/model.torch', 'wb') as f:
        torch.save(results, f)

### Evaluate a representative model

- Input the repetition `r` to load the corresponding trained model.


- Load and process the experimental data.


- Plot the training history.


- Make predictions on all synthetic and experimental data.


- Plot example reconstructions of synthetic data in each error quartile, and reconstructions of experimental data.


- Visualize the latent space along 2 dimensions.


- Plot performance of regressors, if present.

In [ ]:
# repetition to evaluate
r = 0
image_dir = model_dir + '/model_' + str(r)
model, _, _ = init_model(model_name, height, width, num_features, kwargs, device, scaler=scaler)
model.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['state'])
model.eval();

In [ ]:
# plot training history
dynamics = torch.load(image_dir + '/model.torch', map_location='cpu')['dynamics']
plot_history(image_dir, dynamics, logscale=False)

In [ ]:
# plot 2D kernels
plot_weights(model)

In [ ]:
# predict on synthetic data
df = get_predictions(model, data_loaders, ['test'], device, height, width, num_features, kwargs,
                     model_name, y_ids, scaler)
if model_name == 'cvae': df = convert_units(df, M, y_header, y_columns)

In [ ]:
# predict on experimental data
x_exp, dx_exp = process_exp(sample_name, exp_names, q, x_moms)
x_exp = torch.from_numpy(x_exp).unsqueeze(1)
df_exp = get_predictions_exp(model, x_exp, dx_exp, exp_names, device, height, width, num_features, kwargs,
                             model_name, y_ids, scaler)
if model_name == 'cvae':
    df_exp = convert_units(df_exp, M, y_header)
    
    # plot predictions
    fig, ax = plt.subplots(2, len(y_header)//2 + len(y_header)%2, figsize=(15,5.5))
    ax = ax.ravel()
    if len(exp_names) > 1: norm = plt.Normalize(vmin=0, vmax=len(exp_names)-1)
    else: norm = plt.Normalize(vmin=0, vmax=1)
    
    y_min = np.stack(df['y_true_'].values).min(axis=0)[y_ids]
    y_max = np.stack(df['y_true_'].values).max(axis=0)[y_ids]

    for i in range(len(exp_names)):
        dict_exp = dict(zip(y_header, df_exp.iloc[i]['y_pred_']))
        for j, (k, v) in enumerate(dict_exp.items()):
            ax[j].scatter(0, v, s=48, color=cmap_temp(norm(i)), ec='black')
        if i == 0:
            for j, (l, k) in enumerate(zip(y_labels, y_units_)):
                ax[j].set_xticks([])
                if len(k) > 10: y_title = l + '\n' + k
                else: y_title = l + ' ' + k
                ax[j].set_title(y_title, ha='center', loc='right', fontsize=fontsize)
                ax[j].set_ylim([y_min[j] - 0.05*y_max[j], y_max[j]])
                ax[j].tick_params(axis='y', labelsize=fontsize-1)
                ax[j].yaxis.tick_right()
                ax[j].spines['left'].set_visible(False)
                ax[j].spines['top'].set_visible(False)
                ax[j].spines['bottom'].set_visible(False)
    for i in range(j+1,len(y_header)+1):
        try: ax[i].remove()
        except: break
    
fig.subplots_adjust(hspace=0.5, wspace=2.)

In [ ]:
scale = 0
ncol = 2
show_prox = True
for i in range(len(df_exp)):
    y_pred = df_exp.iloc[i]['y_pred'].copy()
    tag = df_exp.iloc[i]['set']
    if i == 0: y_lims = plot_SLD(image_dir, sample_name, q, y_pred, y_header, tag, scale,
                                 show_prox=show_prox, ncol=ncol)
    else: plot_SLD(image_dir, sample_name, q, y_pred, y_header, tag, scale, y_lims, show_prox, ncol)

In [ ]:
# plot example reconstructions
plot_decoded(image_dir, np.stack(df.loc[df['set']=='test', 'x_pred'].values),
             np.stack(df.loc[df['set']=='test', 'x_true'].values),
             np.stack(df.loc[df['set']=='test', 'x_mse'].values), 'test',
             df_exp['x_mse'].values, exp_names)

In [ ]:
# plot reconstructed experiment
plot_decoded_exp(image_dir, np.stack(df_exp['x_pred'].values), sample_name, exp_names, q, x_moms)

In [ ]:
# visualize separability of latent space
plot_latent_representation(image_dir, np.stack(df.loc[df['set']=='test', 'z'].values),
                           np.stack(df.loc[df['set']=='test', 'y_true_'].values),
                           y_ids, y_labels, y_units_, 'encoded_test', np.stack(df_exp['z'].values), exp_names)

In [ ]:
# plot histogram of regressor predictions
plot_predicted(image_dir, np.stack(df.loc[df['set']=='test', 'y_pred_'].values),
               np.stack(df.loc[df['set']=='test', 'y_true_'].values), y_ids, y_labels, y_units_)

### Evaluate all models

- Compile the training history and experiment predictions on all model repetitions.


- Plot the training history statistics.


- Plot regression statistics on experimental data.

In [ ]:
# predict on all models to compute statistics
if model_name == 'cvae':
    x_exp, dx_exp = process_exp(sample_name, exp_names, q, x_moms)
    x_exp = torch.from_numpy(x_exp).unsqueeze(1)

    dynamics = []
    for r in range(reps):
        # load model
        image_dir = model_dir + '/model_' + str(r)
        model, _, _ = init_model(model_name, height, width, num_features, kwargs, device, scaler=scaler)
        model.load_state_dict(torch.load(image_dir + '/model.torch', map_location=device)['state'])
        model.eval()

        # get training history
        dynamics += [torch.load(image_dir + '/model.torch', map_location='cpu')['dynamics']]

        # predict on synthetic data
        df = get_predictions(model, data_loaders, ['valid'], device, height, width, num_features, kwargs,
                             model_name, y_ids, scaler)
        df = convert_units(df, M, y_header, y_columns)
        
        y_true = np.stack(df.loc[df['set']=='valid', 'y_true_'].values)
        y_pred = np.stack(df.loc[df['set']=='valid', 'y_pred_'].values)
    
        k = y_header.index('d_prox')
        d_prox_th = get_threshold(image_dir, y_true[:,y_ids[k]], y_pred[:,k], y_labels[k], y_units_[k], 'd_prox')
        
        k = y_header.index('magn_prox')
        m_prox_th = get_threshold(image_dir, y_true[:,y_ids[k]], y_pred[:,k], y_labels[k], y_units_[k], 'magn_prox')
        
        if 'd_iAFM' in y_header:
            k = y_header.index('d_iAFM')
            d_iAFM_th = get_threshold(image_dir, y_true[:,y_ids[k]], y_pred[:,k], y_labels[k], y_units_[k], 'd_iAFM')
            
            k = y_header.index('magn_iAFM')
            m_iAFM_th = get_threshold(image_dir, y_true[:,y_ids[k]], y_pred[:,k], y_labels[k], y_units_[k], 'magn_iAFM')

        if r > 0:
            df_ = get_predictions_exp(model, x_exp, dx_exp, exp_names, device, height, width, num_features, kwargs,
                                      model_name, y_ids, scaler)
            df_ = convert_units(df_, M, y_header)
            df_['model'] = r; df_['d_prox_th'] = d_prox_th; df_['m_prox_th'] = m_prox_th
            try: df_['d_iAFM_th'] = d_iAFM_th
            except: pass
            else: df_['m_iAFM_th'] = m_iAFM_th
            df_exp = df_exp.append(df_, ignore_index=True)

        else:
            # predict on experimental data
            df_exp = get_predictions_exp(model, x_exp, dx_exp, exp_names, device, height, width, num_features, kwargs,
                                         model_name, y_ids, scaler)
            df_exp = convert_units(df_exp, M, y_header)
            df_exp['model'] = r; df_exp['d_prox_th'] = d_prox_th; df_exp['m_prox_th'] = m_prox_th
            try: df_exp['d_iAFM_th'] = d_iAFM_th
            except: pass
            else: df_exp['m_iAFM_th'] = m_iAFM_th

In [ ]:
# plot training history statistics
plot_history_statistics(model_dir, dynamics, logscale=False)

In [ ]:
y_min = np.stack(df['y_true_'].values).min(axis=0)[y_ids]
y_max = np.stack(df['y_true_'].values).max(axis=0)[y_ids]

for i, y_name in enumerate(y_header):
    if y_name == 'd_prox': y_th = np.mean(reject_outliers(df_exp['d_prox_th'].values))
    elif (y_name == 'magn_prox') or (y_name == 'magn_FM'): y_th = np.mean(reject_outliers(df_exp['m_prox_th'].values))
    elif y_name == 'd_iAFM': y_th = np.mean(reject_outliers(df_exp['d_iAFM_th'].values))
    elif y_name == 'magn_iAFM': y_th = np.mean(reject_outliers(df_exp['m_iAFM_th'].values))
    else: y_th = None
    ylims = [y_min[i], y_max[i]]
    plot_exp_statistics(model_dir, df_exp, reps, y_name, y_header, y_labels, y_units_, y_th=y_th, y_lims=ylims)